In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('CRDW_Registry_IndexDx_2020_Jul23.csv')

In [4]:
total_n = len(data.index)
data = data[~pd.isnull(data.Age)]
print('filtered out {} entries without a valid age'.format(total_n - len(data.index)))

filtered out 46 entries without a valid age


In [5]:
total_n = len(data.index)
data = data[data.raceethnic != '98']
print('filtered out {} entries without a valid race/ethnic entry'.format(total_n - len(data.index)))

filtered out 1 entries without a valid race/ethnic entry


In [6]:
total_n = len(data.index)
data.mrn = pd.to_numeric(data.mrn, errors='coerce')
data = data[~pd.isnull(data.mrn)]
print('filtered out {} entries without a valid MRN'.format(total_n - len(data.index)))

filtered out 1 entries without a valid MRN


In [7]:
total_n = len(data.index)
data = data[~pd.isnull(data.stage1)]
print('filtered out {} entries without a valid stage at diagnosis'.format(total_n - len(data.index)))

filtered out 85 entries without a valid stage at diagnosis


In [8]:
data.mrn = data.mrn.astype(int)
data.Age = data.Age.astype(int)
data.stage1 = data.stage1.astype(int)
data.her2 = pd.to_numeric(data.her2, errors='coerce')

In [9]:
data.binned_age = pd.cut(data.Age, bins=[0, 39, 49, 59, 69, data.Age.max()])
data.binned_age.value_counts(sort=False)

(0, 39]      432
(39, 49]    1013
(49, 59]    1129
(59, 69]     964
(69, 99]     832
Name: Age, dtype: int64

In [10]:
race_groups = {
    'White': ['White'],
    'Black': ['Black'],
    'Asian / Pacific Islander': [
        'Asian Indian', 
        'Asian Indian or Pakistani, NOS', 
        'Filipino', 'Chinese', 
        'Other Asian, including Asian/Oriental, NOS',
        'Japanese',
        'Korean',
        'Hawaiian',
        'Vietnamese',
        'Thai (effective with 1994 diagnoses)',
        'Pakistani'
    ],
    'Other': ['Other', 'Unknown']
}

for label, group in race_groups.items():
    print(label, len(data[data.race_1.str.contains('|'.join(group))]))

White 2439
Black 1680
Asian / Pacific Islander 175
Other 85


In [11]:
print('Hispanic: {}, non-Hispanic: {}'.format(
    len(data[data.raceethnic == 'Hispanic']),
    len(data[data.raceethnic != 'Hispanic'])
    )
)

Hispanic: 115, non-Hispanic: 4255


In [12]:
data.stage1.value_counts().sort_index()

0     678
1    1657
2    1410
3     490
4     135
Name: stage1, dtype: int64

In [13]:
data.hr_pos = (data.ER1 == 1) | (data.PR1 == 1)
data.hr_neg = (data.ER1 == 0) | (data.PR1 == 0)

In [14]:
print('DCIS: {}\nHR+/HER2+: {}\nHR+/HER2-: {}\nHR-/HER2+: {}\nHR-/HER2-: {}\nUnknown: {}'.format(
    len(data[data.stage1 == 0]),
    len(data[(data.stage1 != 0) & (data.hr_pos == True) & (data.her2 == 1)]),
    len(data[(data.stage1 != 0) & (data.hr_pos == True) & (data.her2 == 0)]),
    len(data[(data.stage1 != 0) & (data.hr_pos == False) & (data.her2 == 1)]),
    len(data[(data.stage1 != 0) & (data.hr_pos == False) & (data.her2 == 0)]),
    len(data[(data.stage1 != 0) & (pd.isnull(data.ER1) | pd.isnull(data.PR1) | pd.isnull(data.her2))])
    )
)


DCIS: 678
HR+/HER2+: 329
HR+/HER2-: 1969
HR-/HER2+: 236
HR-/HER2-: 745
Unknown: 636


In [15]:
mammo_data = pd.read_excel('DR_227575_COHORT-20201106.xlsx', engine='openpyxl')

In [16]:
mammo_data['MRN'] = mammo_data['MRN'].astype(int)

In [17]:
merged_data = data.merge(mammo_data, left_on='mrn', right_on='MRN')

In [18]:
len(merged_data)

6